# Process Excel Survey Data

In [ ]:
import pandas as pd
# from datetime import datetime

# Set display options for better viewing
pd.set_option('display.max_columns', 17)
pd.set_option('display.max_colwidth', 40)
# pd.set_option('display.width', 10)

In [ ]:
# Load the Excel file
excel_file = 'moni_data_2025-09-28.xlsx'

# Read the Excel file - you can specify sheet name if needed
# df = pd.read_excel(excel_file, sheet_name='Sheet1')  # or specify the exact sheet name
df = pd.read_excel(excel_file)

print(f"Loaded Excel file: {excel_file}")
print(f"Shape: {df.shape}")

In [ ]:
# Manually map verbose column names to short, intuitive snake_case names

column_map = {
    "First Name": "first_name",
    "Last Name": "last_name",
    "Email Address": "email",
    "Office/Work Phone number (please provide with area code) Eg: +1 for USA, +91 for India": "phone",
    "Link to your Work Website": "website",
    "Name of your institution(s)/work place(s)": "institution",
    "Address of your institution(s)/work place(s)": "address",
    " Language(s) are able to provide genetics/genetic counsel(l)ing services in": "languages",
    "Specialties": "specialties"
}

# Only keep columns that are in the mapping
df = df[[col for col in column_map.keys() if col in df.columns]].copy()

# Rename columns in the dataframe
df = df.rename(columns=column_map)
df.head()

In [ ]:
import validators

# Clean up emails: strip, lowercase, remove commas and spaces
emails_cleaned = (
    df['email']
    .astype(str)
    .str.strip()
    .str.lower()
    .str.replace(',', '', regex=False)
    .str.replace(' ', '', regex=False)
)

# If multiple emails in a cell, split and take the first valid one
def first_valid_email(cell):
    for e in str(cell).split(';'):
        e = e.strip()
        try:
            if validators.email(e):
                return e
        except Exception:
            continue
    return None

df['email'] = emails_cleaned.apply(first_valid_email)

# Show any still-invalid emails
def is_invalid_email(x):
    try:
        return not bool(validators.email(x))
    except Exception:
        return True

invalid_emails = df['email'][df['email'].fillna('').apply(is_invalid_email)]
print("Invalid email addresses after cleaning:")
invalid_emails.value_counts()

In [ ]:
df.email.sample(10)

In [ ]:
import matplotlib.pyplot as plt

# Show bar chart of non-null counts per column (info equivalent)
non_null_counts = df.notnull().sum()
plt.figure(figsize=(10, 5))
non_null_counts.plot(kind='bar')
plt.title('Non-null Value Count per Column')
plt.ylabel('Count')
plt.xlabel('Column')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Show bar charts of basic statistics (describe equivalent) for numeric columns
desc = df.describe().T
if not desc.empty:
    stats_to_plot = ['mean', 'std', 'min', 'max']
    available_stats = [stat for stat in stats_to_plot if stat in desc.columns]
    if available_stats:
        desc[available_stats].plot(kind='bar', subplots=True, layout=(2,2), figsize=(12,8), legend=False)
        plt.suptitle('Basic Statistics for Numeric Columns')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()
    else:
        print("No numeric statistics ('mean', 'std', 'min', 'max') available to plot.")
else:
    print("No numeric columns to describe.")


In [ ]:
# Clean up selected fields: strip whitespace, collapse spaces, and handle newlines in address
fields_to_clean = ['first_name', 'last_name', 'institution', 'address']
for col in fields_to_clean:
    if col in df.columns:
        s = df[col].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)
        if col == 'address':
            s = s.str.replace(r'\n+', ', ', regex=True)
        df[col] = s

df.head(10)

In [ ]:
import numpy as np
import re
import validators

non_url_responses = {
    'prefer not to say', 'not available', 'i do not have a work website',
    'none', 'na', 'n/a', '', 'being updated', 'retired', 'n.a.', 'n.a', 'n.a'
}

# Stats counters
website_stats = {
    'set_to_nan': 0,
    'truncated': 0,
    'fixed_space_split': 0,
    'normalized_www': 0,
    'extracted_url': 0,
    'removed_trailing_slash': 0,
    'prepended_https': 0,
    'could_not_parse': 0,
}

def clean_website_stats(val):
    """
    Clean website field, keep stats of changes, lowercase before processing.
    Only return the first valid URL or domain. Fixes edge case where a space splits a domain.
    """
    if pd.isnull(val):
        return np.nan
    s = str(val).strip().lower()
    if s in non_url_responses:
        website_stats['set_to_nan'] += 1
        return np.nan

    # Remove trailing punctuation and spaces
    s = s.rstrip(' .,/;:\n\t')

    # If there are multiple entries separated by ; or newline, only keep the first
    if re.search(r'[;\n]', s):
        first = re.split(r'[;\n]', s)[0].strip()
        website_stats['truncated'] += 1
    else:
        first = s

    first = first.strip().lower()
    if first in non_url_responses:
        website_stats['set_to_nan'] += 1
        return np.nan

    # Remove trailing punctuation and spaces again after truncation
    first = first.rstrip(' .,/;:\n\t')

    # Fix edge case: if the string starts with http(s):// or www. but contains a space, try to join the next word if it looks like a domain
    if re.match(r'^(https?://|www\.)', first) and ' ' in first:
        words = first.split()
        prefix_match = re.match(r'^(https?://|www\.)', words[0])
        prefix = prefix_match.group(1) if prefix_match else ''
        rest = ''.join(words)
        if prefix and rest.startswith(prefix):
            rest = rest[len(prefix):]
        candidate = prefix + rest
        domain_pattern = r'^[A-Za-z0-9\-\.]+\.[A-Za-z]{2,}([/\w\-\.\?\=\&\%]*)?$'
        if re.search(domain_pattern, candidate, re.IGNORECASE):
            website_stats['fixed_space_split'] += 1
            first = candidate

    # Try to extract a URL or www. domain (allow for spaces in URLs, but only take up to first space)
    url_match = re.findall(r'((?:https?://|www\.)[^\s,;]+)', first)
    if url_match:
        url = url_match[0]
        url = url.rstrip(' .,/;:\n\t')
        if url.startswith('www.'):
            website_stats['normalized_www'] += 1
            url = 'https://' + url
        elif url != first:
            website_stats['extracted_url'] += 1
        return url.rstrip('/')

    # If it looks like a domain (contains a dot, no spaces), prepend https://
    domain_pattern = r'^[A-Za-z0-9\-\.]+\.[A-Za-z]{2,}([/\w\-\.\?\=\&\%]*)?$'
    if '.' in first and ' ' not in first:
        if first.startswith(('http://', 'https://')):
            if first.endswith('/'):
                website_stats['removed_trailing_slash'] += 1
            return first.rstrip('/')
        if re.match(domain_pattern, first, re.IGNORECASE):
            website_stats['prepended_https'] += 1
            return 'https://' + first.rstrip('/')
    # If it looks like a domain with spaces (e.g. 'ahs.ca/genetics'), try to extract the domain part
    if '.' in first and '/' in first and ' ' in first:
        domain_candidate = first.split(' ')[0]
        if re.match(domain_pattern, domain_candidate, re.IGNORECASE):
            website_stats['prepended_https'] += 1
            return 'https://' + domain_candidate.rstrip('/')

    website_stats['could_not_parse'] += 1
    return first

if 'website' in df.columns:
    orig = df['website'].copy()
    df['website'] = df['website'].apply(clean_website_stats)

    # Show value counts for values that became null
    turned_null = orig[~orig.isnull() & df['website'].isnull()]
    if not turned_null.empty:
        print("Values turned into nulls in 'website':")
        print(turned_null.value_counts())

    # Now check for validity using validators
    cleaned = df['website'].dropna().unique()
    invalid = []
    for val in cleaned:
        # Accept LinkedIn and Google as special cases
        if isinstance(val, str) and (
            'linkedin.com' in val or
            'google.com' in val or
            val == 'linkedin' or
            val == 'google'
        ):
            continue
        try:
            if validators.url(val) or validators.domain(val):
                continue
        except Exception as e:
            invalid.append(val)
            continue
        invalid.append(val)
    if invalid:
        print("Website values thrown out by validators (these are kept in the data, but flagged as invalid):")
        for v in invalid:
            print("  ", v)
    else:
        print("All cleaned website values are valid.")

    # Print stats summary
    print("\nWebsite cleaning stats:")
    for k, v in website_stats.items():
        print(f"  {k}: {v}")

In [ ]:
df.website.sample(15).values

In [ ]:
import re

def clean_phone(val):
    if pd.isnull(val):
        return None
    val_str = str(val).strip()
    if val_str.lower() == 'prefer not to say' or val_str == '':
        return None

    # If value contains a slash, try to extract the phone number part
    if '/' in val_str:
        # Split on slash and look for a phone-like pattern in each part
        parts = [p.strip() for p in val_str.split('/')]
        phone_candidate = None
        for part in parts:
            # Look for a sequence of digits, possibly with +, spaces, dashes, or parentheses
            if re.search(r'(\+?\d[\d\s\-\(\)]{5,})', part):
                phone_candidate = part
                break
        if phone_candidate:
            val_str = phone_candidate

    # Remove any leading/trailing whitespace again
    val_str = val_str.strip()
    # Remove double spaces
    val_str = re.sub(r' {2,}', ' ', val_str)
    if val_str == '':
        return None
    return val_str

df['phone'] = df['phone'].apply(clean_phone)

In [ ]:
df.phone.sample(20).to_list()

In [ ]:
df.sample(10)

In [ ]:
import re

def uses_interpreters_func(val):
    if pd.isnull(val):
        return False

    val_str = str(val).lower()

    # Phrases that indicate interpreter/translation services
    interpreter_patterns = [
        r'interpret(er|ation)( services| present| available)?',
        r'with use of',
        r'with provided interpreter',
        r'others? with (provided )?interpreter( services)?',
        r'all other languages with interpretation services',
        r'we also use interpreters?',
        r'globo interpreter',
        r'translation',
        r'interpreter present',
        r'other languages with an interpreter present',
    ]
    for pat in interpreter_patterns:
        if re.search(pat, val_str):
            return True
    return False

def clean_languages(val):
    if pd.isnull(val):
        return None

    # Convert to string and strip whitespace
    val = str(val).strip()

    # Remove parentheses and their contents
    val = re.sub(r'\([^)]*\)', '', val)

    # Remove "prefer not to say" and similar
    if re.search(r'prefer not to say', val, re.IGNORECASE):
        return None

    # Remove phrases indicating interpreter/translation services
    val = re.sub(
        r'(all other languages with interpretation services|others? with (provided )?interpreter( services)?|interpretation services available|we also use interpreters?|limited [a-z]+|with use of [A-Za-z ]+|other languages with an interpreter present|interpreter present|translation|globo interpreter)',
        '', val, flags=re.IGNORECASE
    )

    # Remove trailing/leading whitespace again after phrase removal
    val = val.strip()

    # Normalize separators: replace ; | / and newlines with commas
    val = re.sub(r'[;|/]', ',', val)
    val = val.replace('\n', ',')

    # Replace " and ", " & ", " and", "&", "and " (with or without spaces) with comma
    val = re.sub(r'(\s+and\s+|\s+&\s+|^and\s+|\s+and$|^&\s+|\s+&$)', ',', val, flags=re.IGNORECASE)
    val = re.sub(r'^(and|&)\s+', ',', val, flags=re.IGNORECASE)
    val = re.sub(r'\s+(and|&)$', ',', val, flags=re.IGNORECASE)

    # Remove periods (special characters) except those in abbreviations (e.g., "U.S.A." becomes "USA")
    val = re.sub(r'\.', '', val)

    # Remove any other special characters except commas, letters, and spaces
    val = re.sub(r'[^a-zA-Z,\s]', '', val)

    # Remove extra spaces
    val = re.sub(r'\s+', ' ', val)

    # Remove empty commas and leading/trailing commas
    val = re.sub(r',+', ',', val)
    val = val.strip(',')
    val = val.strip()

    # Sometimes people write languages without commas, e.g. "Kannada English Hindi"
    # If there are 2+ words with no commas, split them
    # First, split by comma, then further split by space if the entry contains multiple words and no comma
    langs = []
    for chunk in val.split(','):
        chunk = chunk.strip()
        if not chunk:
            continue
        # If chunk contains multiple words, split them
        words = chunk.split()
        if len(words) > 1:
            langs.extend(words)
        else:
            langs.append(chunk)

    # Remove any that are just "other", "others", "interpreter", "interpreter present", "interpretation", "translation", "services", "provided", "available", "languages", "with", "an", "present", "some"
    ignore = {
        'other', 'others', 'interpreter', 'interpreter present', 'interpretation', 'translation',
        'services', 'provided', 'available', 'languages', 'with', 'an', 'present', 'some'
    }
    langs = [l for l in langs if l.lower() not in ignore and l.strip()]

    # Do NOT try to interpret shortcodes for languages or filter out single-character codes

    # Capitalize each language (special handling for multi-word languages is not needed now)
    langs = [l.strip().capitalize() for l in langs]

    # Remove duplicates while preserving order (case-insensitive)
    seen = set()
    langs_clean = []
    for l in langs:
        l_key = l.lower()
        if l_key not in seen:
            langs_clean.append(l)
            seen.add(l_key)

    # Always sort the languages alphabetically (case-insensitive)
    langs_clean_sorted = sorted(langs_clean, key=lambda x: x.lower())

    return ', '.join(langs_clean_sorted) if langs_clean_sorted else None

# Keep the original languages column for comparison
df['languages_original'] = df['languages']

df['uses_interpreters'] = df['languages'].apply(uses_interpreters_func)
df['languages'] = df['languages'].apply(clean_languages)

In [ ]:
df['email languages_original languages uses_interpreters'.split()].sample(20)

In [ ]:
del df['languages_original']

In [ ]:
df.languages.value_counts()

In [ ]:
df.sample(10)

In [ ]:
# Check for missing values
print("Missing values per column:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

if missing_values.sum() == 0:
    print("No missing values found!")


In [ ]:
# Move "specialties" to the last column
cols = [col for col in df.columns if col != 'specialties'] + ['specialties']
df = df[cols].copy()
df

In [ ]:
# Rename columns before export
df_export = df.rename(columns={
    'first_name': 'name_first',
    'last_name': 'name_last',
    'email': 'email',
    'phone': 'phone_work',
    'website': 'work_website',
    'institution': 'work_institution',
    'address': 'work_address',
    'languages': 'language_spoken',
    'uses_interpreters': 'uses_interpreters',
    'specialties': 'specialties'
})

# Export CSV with quoting to ensure Excel/Google Sheets parse fields correctly
import csv
df_export.to_csv(
    "data_for_geocoding.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # Quote all fields to prevent misparsing
    encoding="utf-8-sig"    # Add BOM for Excel compatibility
)

In [ ]:
df_export.query("uses_interpreters == True")